In [ ]:
import pandas as pd
import numpy as np
import re
import os
import datetime
print('导入成功')
pd.set_option('display.max_columns',200)
pd.set_option('max_row',200)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

In [ ]:
# #身份证编码规则：
# 1、新版中国的居民身份证有18位；
# 2、其中前17位是信息码，最后1位是校验码；
# 3、每位信息码可以是0-9的数字，而校验码可以是0-9或X，其中X表示10；
# 4、第17位数字代表“性别”，奇数为“男”，偶数为“女”；
# 5、第1-2位数字表示“所在地身份（或自治区、直辖市）”；
# 6、第3-4位数字表示“所在地的市（或州）”；
# 7、第5-6位数字表示“所在地的县（或县级市）”；
# 8、最后4位的前两位表示“当地派出所的编号”；
# 9、排列顺序从左至右依次为:六位数字地址码，八位数字出生日期码，三位数字顺序码和一位数字校验码。
# 10、身份证第18位（校验码）的计算方法：
# （a）设18位身份证号序列从左到右为: 
# a[0], a[1], a[2], a[3], ..., a[16], a[17]其中a[i]表示第i位数字，i=0,1,2,...,17，（如果最后一位（校验位）是X，则a[17]=10）
# (b)每一位被赋予一个“权值”，其中，第i位的权值w[i]的计算方法是： 
# w[i] = 2**(17-i) % 11，其中，i=0,1,2,3,...,17，运算符按Python惯例：x**y表示x的y次方，x%y表示x除以y的余数。
# 最终得到从第一位到第十七位的权重系数分别为：[7, 9, 10, 5, 8, 4, 2, 1, 6, 3, 7, 9, 10, 5, 8, 4, 2]
# (c)将这17位数字和系数相乘的结果相加,用加出来和除以11，看余数是多少？
# 即：(a[0]*w[0] + a[1]*w[1] + a[2]*w[2] + ... + a[16]*w[16]) % 11 
# （d）余数只可能有0－1－2－3－4－5－6－7－8－9－10这11个数字,
# 其分别对应的最后一位身份证的号码为1－0－X－9－8－7－6－5－4－3－2。

In [ ]:
df=pd.read_excel(r'D:\000-mine\richang2019\临时\0719\订单明细.xlsx',sep='\t',index_col=None)
df.info()

In [ ]:
sf_df=pd.DataFrame(df,index=df.index,columns=['订单编号','订单状态','订单生成时间','物流单号',
                                              '交易完成时间','入网用户姓名','入网身份证号','入网手机号',
                                             '收货人姓名','所在省 / 市 / 县','号码归属地','销售品编号',
                                             '销售品名称'])
#“所在省 / 市 / 县”——配送地址，‘号码归属地’——手机卡归属地,"入网身份证号"——籍贯的省市，“收货电话”——号码归属省，
#4个地址一致性：超三个为高风险用户。
# print(sf_df[sf_df['入网身份证号'].isnull()])   #校验“入网身份证号”为空的行数据

In [ ]:
sf_df.dropna(subset=['入网身份证号'],inplace=True)  #删除“入网身份证号”为空的行数据
sf_df.reset_index(drop=True,inplace=True)   #重置索引，并删除原来的索引

In [ ]:
# 方法一：使用for循环可以实现将“奇偶数”表示的性别转为“男女”格式，但是时间复杂度太高
# # sf_df['性别']=sf_df['入网身份证号'].str.slice(16,17)
# # for i in range(len(sf_df['性别'])):
# #     if int(sf_df.loc[i,['性别']]) % 2==0:
# #         sf_df.loc[i,['性别']]="女"
# #     else:
# #         sf_df.loc[i,['性别']]="男"
# 方法二：先将“奇偶数”表示的性别列取出，后将object类型数据转为数值型，再直接调用函数，速度很快
# help(df.infer_objects)  #该命令可调出关于将object类型数据转为何种数据类型的语法帮助

In [ ]:
#根据“订单生成时间”到“交易完成时间”之差即：“交易周期”列
#Dataframe中的时间是不能直接进行相加减的,所以需要先用pandas的to_datetime()方法，转化成时间格式进行加减，然后再转换成df格式
#delta=df1['Time_end']-df1['Time_start'] #直接报错TypeError: unsupported operand type(s) for -: 'str' and 'str'
#日期相减变为小时；变为天的话将h替换为D即可：
sf_df['交易周期']=pd.DataFrame((pd.to_datetime(sf_df['交易完成时间'])-
                            pd.to_datetime(sf_df['订单生成时间'])).values/np.timedelta64(1,'h'))
print(sf_df['交易周期'].tail())

In [ ]:
sf_df['性别']=sf_df['入网身份证号'].str.slice(16,17)
sf_df['性别'] = pd.to_numeric(sf_df['性别'])   #这个函数是将object转为数值型
def get_sex(str1):
    #查看性别
    if str1 % 2 ==0:
        return '女'
    else:
        return '男'

sf_df['性别'] =  sf_df['性别'].apply(get_sex)
# print(sf_df.info())

In [ ]:
#对用户根据年龄段进行分类
sf_df['年龄'] =sf_df['入网身份证号'].str.slice(6,10)   #从“入网身份证号”中提取“年龄信息”
sf_df['年龄']=pd.to_numeric(sf_df['年龄'])  ##这个函数是将object转为数值型
# print(sf_df.loc[:10,['年龄分级']])
now_year=pd.datetime.now().year  #获取当前的年份
# print("当前时间是:",now_year)
sf_df['年龄']=now_year-sf_df['年龄']  #当前年份-出生年龄得到，年龄分级
# print(sf_df.loc[:10,['年龄分级']])
def get_year_group(str1):  #定义年龄分级函数
    if str1 <= 18:
        return "少年"
    elif str1 < 30:
        return "弱冠"
    elif str1 < 40:
        return "而立"
    elif str1 < 50:
        return "不惑"
    elif str1 < 60:
        return "知命"
    elif str1 < 70:
        return "花甲"
    else:
        return "老年"

sf_df['年龄分级']=sf_df['年龄'].apply(get_year_group)  #调用年龄分级函数，用df[col_name].apply(function_name)
#将'号码归属地'的列拆分为“号码归属省”和“号码归属市”两列   #即分析用户的地域特征
# sf_df[['所在省 / 市 / 县','号码归属地']].head(100)   
# df=sf_df['号码归属地'].str.split('/')  #该种方法也可实现，不过时间复杂度较高
df_add=pd.DataFrame((x.split('/') for x in sf_df['号码归属地']),
                    index=sf_df.index,columns=['号码归属省','号码归属市'])
# print(df_add.tail())
sf_df=sf_df.join(df_add)
sf_df.drop(['号码归属地'],axis=1,inplace=True)

In [ ]:
# "发货量"为“物流单号”不为空的订单，“激活量”为“订单状态”为“交易完成”的订单
# sf_df['物流单号'].astype(str).dtype   #此种方法将object类型转为str类型失效
sf_df_fahuo=sf_df[sf_df['物流单号'].notnull()]   #筛选出“物流单号”不为空的行数据即：“发货量”
print(sf_df_fahuo.info())
#新增“激活与否”列，根据“订单状态”为“交易完成”的为“已激活订单”赋值为1，“未激活订单”赋值为0；
sf_df_fahuo['激活与否']=None
def jh_decide(str1):
    if(str1=="交易完成"):
        return 1
    else:
        return 0
    
sf_df_fahuo['激活与否']=sf_df_fahuo['订单状态'].apply(jh_decide)
# print(sf_df_fahuo.loc[:100,['订单状态','激活与否']])   #验证新增列“激活与否”与“订单状态”对应

In [ ]:
#匹配产品标卡
biaoka=pd.read_excel(r'D:\000-mine\richang2019\原始数据集\产品标卡0709.xlsx',sep='\t',index_col=None)
biaoka.drop_duplicates(subset=['销售品编号'],keep='last',inplace=True)  #去重
biaoka.drop(['销售品名称'],axis=1,inplace=True)
sf_df_fahuo=pd.merge(sf_df_fahuo,biaoka,how='left',on='销售品编号')
print(biaoka.info())

In [ ]:
print(sf_df_fahuo.info())

In [ ]:
#part01 分析“激活与否”与“性别”、“年龄分级”、“产品分类”、“号码归属省”、“交易周期”各特征变量的关系
#特征相关性分析，spearman用于离散型变量
print(sf_df_fahuo['激活与否'].corr(sf_df_fahuo['性别'],method='spearman'))  #两列之间的相关度计算
print(sf_df_fahuo['激活与否'].corr(sf_df_fahuo['年龄分级'],method='spearman')) 
print(sf_df_fahuo['激活与否'].corr(sf_df_fahuo['号码归属省'],method='spearman')) 
print(sf_df_fahuo['激活与否'].corr(sf_df_fahuo['产品分类'],method='spearman'))
print(sf_df_fahuo['激活与否'].corr(sf_df_fahuo['交易周期'],method='spearman'))
print("="*50)
print(sf_df_fahuo['产品分类'].corr(sf_df_fahuo['性别'],method='spearman'))
print(sf_df_fahuo['产品分类'].corr(sf_df_fahuo['年龄分级'],method='spearman'))
print(sf_df_fahuo['产品分类'].corr(sf_df_fahuo['号码归属省'],method='spearman'))
print(sf_df_fahuo['产品分类'].corr(sf_df_fahuo['交易周期'],method='spearman'))

In [ ]:
#原始数据的可视化分析
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format='retina'  #在屏幕上显示高清图片
%matplotlib inline
import matplotlib as mpl            #解决中文不识别的问题
import matplotlib.ticker as ticker
mpl.rcParams['font.sans-serif']=['KaiTi']  
mpl.rcParams['font.serif']=['KaiTi']
mpl.rcParams['axes.unicode_minus']=False  # 解决保存图像是负号'-'显示为方块的问题,或者转换负号为字符串

In [ ]:
sf_df_fahuo['激活与否'].value_counts()

In [ ]:
#订单的“激活与否”的总体分布情况——“1”表示“激活”，“0”表示“未激活”
f,ax=plt.subplots(1,1,figsize=(10,10))
sf_df_fahuo['激活与否'].value_counts().plot.pie(autopct='%1.2f%%')
ax.set_title("激活与否的分布百分比")

In [ ]:
#不同年龄段的分布情况
print(sf_df_fahuo['年龄'].describe()) 
#分析总体的年龄分布
plt.figure(figsize=(10,3))
# plt.subplot(121)
sf_df_fahuo.boxplot(column='年龄',showfliers=False)   #用户年龄分布的“箱线图”，均值为30岁，
# plt.subplot(122)
facet=sns.FacetGrid(sf_df_fahuo,hue="激活与否",aspect=3)  #先sns.FacetGrid画出轮廓
facet.map(sns.kdeplot,'年龄',shade=True)   #然后用map填充内容，此处为kdeplot(核密度估计图)，

# 通过核密度估计图可以比较直观的看出数据样本本身的分布特征   #分析可知男、女的“激活与否”的“年龄”密度分布基本一致
facet.set(xlim=(0,sf_df_fahuo['年龄'].max()))
facet.add_legend()   #图例legend设置，legend语法参数如下: matplotlib.pyplot.legend(*args, **kwargs)

#"年龄分级"与“激活与否”的关系的条形图
nj_jh=sf_df_fahuo[['年龄分级','激活与否']].groupby(['年龄分级'],as_index=False).mean()
nj_jh.set_index('年龄分级',inplace=True)
nj_jh.sort_values("激活与否",ascending=False).plot.bar()  #观察可知：年龄越大，人越趋于理性，激活率越高
import pylab as pl
pl.xticks(rotation=45)    #matplotlib画图，x轴标签旋转45度
# nj_jh.set_title("年龄分级与激活与否的分布关系")  #该方法失效

In [ ]:
#分析“产品分类”与“激活与否”的分布关系，图为“产品分类”按“激活率”大小降序排列
cp_jh_ys_c=sf_df_fahuo[['产品分类','激活与否']].groupby(['产品分类'],as_index=False).count()   #统计订单量
cp_jh_ys_c=cp_jh_ys_c[cp_jh_ys_c['激活与否']>9]  #筛选“订单量”大于9单的卡品
cp_jh_ys_c.set_index('产品分类',inplace=True)
cp_jh_ys_c.rename(columns={'激活与否':'订单量'},inplace=True)
print(cp_jh_ys_c)  #查看筛选后的数据
cp_jh_ys_c.sort_values('订单量',ascending=False).plot.bar()
pl.xticks(rotation=45)
plt.rcParams['figure.figsize'] = (10.0, 6.0) # 设置figure_size尺寸
plt.rcParams['image.interpolation'] = 'nearest'
# #总体"产品分类"与“激活与否”的关系
# cp_jh_ys=sf_df_fahuo[['产品分类','激活与否']].groupby(['产品分类'],as_index=False).agg(['mean', 'count']) #引出复合索引不好处理
cp_jh_ys_m=sf_df_fahuo[['产品分类','激活与否']].groupby(['产品分类'],as_index=False).mean()   #统计激活率
cp_jh_ys_m.set_index('产品分类',inplace=True)
cp_jh_ys_m.rename(columns={'激活与否':'激活率'},inplace=True)
print(cp_jh_ys_m)  #查看筛选后的数据
cp_jh_ys_m.sort_values('激活率',ascending=False).plot.bar()
pl.xticks(rotation=45)    #matplotlib画图，x轴标签旋转45度
plt.rcParams['figure.figsize'] = (10.0, 6.0) # 设置figure_size尺寸
plt.rcParams['image.interpolation'] = 'nearest'

In [ ]:
#分析“交易周期”与“激活与否”的关系
sf_df_fahuo_jy=sf_df_fahuo[sf_df_fahuo['交易周期']<1000]  #“交易周期”大于1000小时的极个别订单过滤掉
print(sf_df_fahuo_jy.describe())

In [ ]:
#总体“性别”与“激活与否”的关系
mpl.rc("figure", figsize=(5,5))   #大小设置
sf_df_fahuo[['性别','激活与否']].groupby(['性别']).mean().plot.bar()   #女性的激活率相对较高
ax.set_title("性别与激活与否的分布关系")
# train_data[['Sex','Survived']].groupby(['Sex']).mean().plot.bar()

In [ ]:
#"交易周期"的频率分布直方图
jy_jh=sf_df_fahuo_jy[['交易周期','激活与否']].groupby('交易周期',as_index=False).count()
#直方图与柱状图外观表现很相似，用来展现连续型数据分布特征的统计图形（柱状图主要展现离散型数据分布）
# jy_jh['交易周期'].hist(bins=200)   #无拟合曲线 
# sns.set_palette("hls") #设置所有图的颜色，使用hls色彩空间
mpl.rc("figure", figsize=(30,10))   #大小设置
sns.distplot(jy_jh['交易周期'],bins=200,kde_kws={"color":"seagreen", "lw":5 },
             hist_kws=dict(edgecolor='k',color='Teal'))  #edgecolor设置柱子边界线，color设置柱子颜色#kde_kws为拟合曲线参数
plt.xlabel('交易周期')
plt.ylabel('订单量')
plt.xlim(-10,400)#设置x轴分布范围
sns.set_context("talk", font_scale=2, rc={'line.linewidth':2.5})  # 使用talk风格，字体大小

In [ ]:
f,ax=plt.subplots(2,2,figsize=(30,24))  #设置图幅的大小和数量
sns.countplot('性别',data=sf_df_fahuo,ax=ax[0,0],hue='激活与否')  #绘制性别的统计图
sns.countplot('年龄分级',data=sf_df_fahuo,ax=ax[0,1],hue='激活与否')  #绘制“年龄级别”的统计图
sns.countplot('号码归属省',data=sf_df_fahuo,ax=ax[1,0],hue="激活与否") #绘制“号码归属省”的统计图
sns.countplot('产品分类',data=sf_df_fahuo,ax=ax[1,1],hue='激活与否')
ax[0,0].set_title("订单量与性别的关系分布")   #标题设置
ax[0,1].set_title("订单量与年龄分级的关系分布")
ax[1,0].set_title("订单量与号码归属省的关系分布")
ax[1,1].set_title("订单量与产品分类的关系分布")
# ax[1,0].set_xticklabels(ax.get_xticklabels(),rotation=-45)
# ax[0,1].set_xlabel('年龄分级',fontsize=12,color='r')   #设置X轴标签的“名称”、字体大小和字体颜色
# ax[0,1].set_ylabel('订单量',fontsize=12,color='r')    #设置Y轴标签的“名称”、字体大小和字体颜色
#设置网格样式
ax[1,0].grid(True, linestyle='-.')
for label in ax[1,0].xaxis.get_ticklabels():
#     label.set_color('red')   #设置x轴刻度的字体的旋转方向
    label.set_rotation(45)
    label.set_fontsize(12)
#     label.set_visible(False)   # x轴标签过于密集的解决方法,根据实际情况隐藏一些点,set_visible(False)是不显示，为True是显示
    label.set_visible(True)
ax[1,1].grid(True,linestyle='-.')

for label in ax[1,1].get_xticklabels():
    label.set_rotation(45)
    label.set_visible(False)
for label in ax[1,1].xaxis.get_ticklabels()[::1]:     # x轴标签过于密集的解决方法,根据实际情况隐藏一些点,[::2]表示间隔2个显示
    label.set_visible(True)

In [ ]:
#单幅图的绘制
# f,ax=plt.subplots(1,1,figsize=(50,30))  #设置图幅的大小和数量
# sns.countplot('号码归属省',data=sf_df,hue="性别")
# ax.set_title("订单量与号码归属省的关系分布")
# # import matplotlib.ticker as ticker
# for label in ax.xaxis.get_ticklabels():
#     label.set_rotation(45)
#     label.set_fontsize(20)
    
# ax.set_xlabel('号码归属省',fontsize=20,color='r')   #设置X轴标签的“名称”、字体大小和字体颜色
# ax.set_ylabel('订单量',fontsize=20,color='r')    #设置Y轴标签的“名称”、字体大小和字体颜色

In [ ]:
#分析“产品分类”与“年龄分级”的关系
sf_df_fahuo.groupby(['产品分类','年龄分级'])['年龄分级'].count()

In [ ]:
#"产品分类"与“年龄”的分布关系
fig, ax = plt.subplots(1, 2, figsize = (30, 25))
sns.violinplot("性别", "年龄", hue="激活与否", data=sf_df_fahuo, split=True, ax=ax[0])
ax[0].set_title('性别 and 年龄 vs 激活与否')
ax[0].set_yticks(range(0, 110, 10))
ax[0].grid(True, linestyle='-')

sns.violinplot("产品分类","年龄",hue="激活与否",data=sf_df_fahuo, split=True, ax=ax[1])
ax[1].set_title("产品分类 and 年龄 vs 激活与否" )
ax[1].set_yticks(range(0,110,10))
ax[1].grid(True,linestyle='-')

In [ ]:
#参数as_index=False，将索引列转为“普通列”
# sf_df_fahuo[['产品分类','年龄分级']].groupby(['年龄分级'],as_index=False).count()
# ax.set_title("产品分类与年龄分级的分布关系")
cp_nj=sf_df_fahuo[['产品分类','年龄分级','订单编号']].groupby(['产品分类','年龄分级'],as_index=False).count()
# sf_df_fahuo.groupby(['产品分类','年龄分级'])['年龄分级'].count()
f,ax=plt.subplots(1,1,figsize=(30,30))  #设置图幅的大小和数量
for label in ax.xaxis.get_ticklabels():
    label.set_rotation(45)
    ax.grid(True,linestyle='-')
sns.barplot(x='产品分类', y='订单编号', hue="年龄分级",data=cp_nj,ci=0)
# ax = sns.barplot(x="day", y="total_bill", hue="sex", data=tips,ci=0)

In [ ]:
print(sf_df_last['年龄分级'].nunique())
print(sf_df_last['产品分类'].nunique())  #统计“产品分类”类不重复项的个数
print(sf_df_last['号码归属省'].nunique())

In [ ]:
#part 02 变量的转换
#变量转换的目的是将数据转换为适用于模型使用的数据，不同模型接受不同类型的数据。
#Scikit-learn要求数据都是数字型numeric，所以我们要将一些非数字型的原始数据转换为数字型numeric。
#定性(Qualitative)转换：此时qualitative variable是一些频繁出现的几个独立变量时，Dummy Variables比较适合使用
sf_df_last=sf_df_fahuo[['订单编号','性别','年龄','年龄分级','产品分类','号码归属省','交易周期','激活与否']]
sf_df_last.tail()

In [ ]:
#方法一：使用factoriz进行编码
sf_df_last['性别']=pd.factorize(sf_df_last['性别'])[0]
#方法二：使用dummies进行编码
# sex_dummies_df=pd.get_dummies(sf_df_last['性别'],prefix=sf_df_last[['性别']].columns[0])
# sf_df_last=pd.concat([sf_df_last,sex_dummies_df],axis=1)

In [ ]:
#描述相关性
#一般的，描述相关性一般使用 pairplot 图和 heatmap 图
sns.set(style='ticks')
pairplot_df=sf_df_fahuo[['激活与否','性别','年龄分级','号码归属省','产品分类']]
sns.pairplot(pairplot_df,hue='激活与否',palette = 'seismic',size=1.2,diag_kind = 'kde',
             diag_kws=dict(shade=True),plot_kws=dict(s=10))


In [ ]:
Correlation=sf_df_fahuo[['交易周期','性别','年龄分级','号码归属省','产品分类']]

colormap = plt.cm.viridis
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(Correlation.astype(float).corr(),linewidths=0.1,vmax=1.0,
            square=True, cmap=colormap, linecolor='white', annot=True)